# Step 1: fine-tune LLM using top result from (fixed) ranker

In [2]:
def step_one(k):
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import IA3Config, TaskType
    import sys
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave, Filter

    sys.stderr = Filter(sys.stderr, r'Bad pipe message') 
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2, timesplit=True)
    dev = dev_loader(batch_size=2, timesplit=True)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    taskllm_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(taskllm_config, "taskllm")
    t5.enable_adapters()

    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")

    with ProgressPrinter('iter', f'{k} loss', f'{k} acc', f'{k} acc (dev)') as printer:
        for iteration in range(2):
            for istrain, (examples, labels) in interleave(train, dev):
                if iteration == 0 or not istrain:
                    with torch.no_grad():
                        inputs = []
                        target = []
        
                        for ex, label in zip(examples, labels):
                            embeddings = train.embed( [ ex['ref1'], ex['ref2'] ] + 
                                                      [ v['title'] 
                                                       for v in ex['profile']
                                                       if v['title'] != ex['title'] 
                                                     ])
                            scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
                            index = torch.topk(scores, dim=0, k=k).indices.to('cpu')
                            for n, oneind in enumerate(index.tolist()):
                                titles = [ f'{ex["profile"][ind]["title"]}' for ind in (oneind,) ]
                                concat_titles = ' and '.join([f'"{v}"' for v in titles])
                                input = train.append_to_title(ex, concat_titles)
                                inputs.append(input)
                                target.append(int(label == train.choices[1]))

                        target = torch.Tensor(target).long().to(device)
                        acc = (taskllm.predict(inputs, augment=train.swap_refs).argmax(dim=1) == target).float().mean().item()
    
                    loss = taskllm.learn(inputs, target, augment=train.swap_refs) if istrain else None
                    printer.addobs(iteration, loss, acc if istrain else None, acc if not istrain else None)

            printer.print()
            printer.autoprint = False
            if iteration == 0:
                taskllm.save_pretrained(f'Time_keq{k}_t5base_step1')

from Fork import SubProcess
with SubProcess() as process: process.parent or step_one(1)

n                  iter       since      1 loss       since       1 acc       since 1 acc (dev)       since      dt (s)
1                     0           0       0.702       0.702           0           0           0           0        1.02
2                     0           0       0.701         0.7           0           0           0           0        1.51
4                     0           0        0.71       0.719       0.125        0.25           0           0        2.52
8                     0           0       0.705       0.698       0.286         0.5           0           0        4.17
16                    0           0       0.696       0.686         0.5        0.75       0.333         0.5        7.36
32                    0           0       0.693       0.691       0.481       0.462         0.5       0.667        14.5
64                    0           0       0.692       0.691       0.471       0.462       0.583       0.667        30.3
128                   0           0     

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [8]:
def learn_ranker(*, max_iteration, rank):
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import IA3Config, TaskType
    import sys
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave, Filter

    sys.stderr = Filter(sys.stderr, r'Bad pipe message') 
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2, double_data=True, timesplit=True)
    dev = dev_loader(batch_size=2, timesplit=True)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter('Time_keq1_t5base_step1', 'taskllm')

    rhat_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(rhat_config, "rhat")
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat")
    
    def reward_augment(inputs):
        import re
        return [ re.sub(r'Ref1: (.*)\nRef2: (.*)\nExtra:',
                        r'Ref1: \2\nRef2: \1\nExtra:',
                        z)
                 for z in inputs ]

    with ProgressPrinter('iter', f'{rank} loss', f'{rank} acc', f'{rank} acc (dev)') as printer:
        for iteration in range(max_iteration):
            for istrain, (examples, labels) in interleave(train, dev):
                if iteration + 1 < max_iteration or not istrain:
                    for ex, label in zip(examples, labels):
                        greedyrewards = []
                        allloss = []
                        with torch.no_grad():
                            embeddings = train.embed( [ ex['ref1'], ex['ref2'] ] + 
                                                      [ v['title'] 
                                                       for v in ex['profile']
                                                       if v['title'] != ex['title'] 
                                                     ])
                            scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
                            index = torch.topk(scores, dim=0, k=rank).indices.to('cpu')
                            prompts = []
                            rhatprompts = []
                            for n, oneind in enumerate(index.tolist()):
                                titles = [ f'{ex["profile"][ind]["title"]}' for ind in (oneind,) ]
                                concat_titles = ' and '.join([f'"{v}"' for v in titles])
                                input = train.append_to_title(ex, concat_titles)
                                prompts.append(input)
                                rhatprompt = f"Title: {ex['title']}\nRef1: {ex['ref1']}\nRef2: {ex['ref2']}\nExtra: {titles[0]}"
                                rhatprompts.append(rhatprompt)
                            
                            guesses = taskllm.predict(prompts, augment=train.swap_refs).argmax(dim=1)
                            target = int(label == train.choices[1])
                            rewards = (guesses == target).float().unsqueeze(1)
                            rhats = rewardpredictor.predict(rhatprompts, augment=reward_augment)
                            greedy = torch.argmax(rhats, dim=0).item()
                            greedyreward = rewards[greedy, 0].item()
                            greedyrewards.append(greedyreward)
                            
                        loss = rewardpredictor.learn(rhatprompts, rewards, augment=reward_augment) if istrain else None
                        allloss.append(loss)

                    greedyacc = torch.Tensor(greedyrewards).float().mean().item()
                    predloss = torch.Tensor(allloss).mean().item() if istrain else None

                    printer.addobs(iteration, predloss, greedyacc if istrain else None, greedyacc if not istrain else None)

            printer.print()
            printer.autoprint = False
            if iteration + 1 < max_iteration:
                rewardpredictor.save_pretrained(f'Time_keq1_t5base_step2_rankeq{rank}_iter{iteration}')

from Fork import SubProcess
with SubProcess() as process: process.parent or learn_ranker(rank=8, max_iteration=8)

n                  iter       since      8 loss       since       8 acc       since 8 acc (dev)       since      dt (s)
1                     0           0       0.686       0.686           1           1           0           0        1.69
2                     0           0       0.669       0.653         0.5           0           0           0        2.76
4                     0           0       0.679       0.689         0.5         0.5           0           0        4.98
8                     0           0       0.693       0.706         0.5         0.5           0           0        9.12
16                    0           0       0.688       0.684         0.6       0.714           1           1        17.5
32                    0           0       0.644       0.596       0.586       0.571       0.667         0.5        34.8
64                    0           0       0.666       0.688       0.603       0.621       0.667       0.667        69.2
128                   0           0     

<class 'KeyboardInterrupt'>


KeyboardInterrupt: 


  File "/tmp/ipykernel_1753555/693645465.py", line 85, in <module>
    with SubProcess() as process: process.parent or learn_ranker(rank=8, max_iteration=8)
  File "/tmp/ipykernel_1753555/693645465.py", line 63, in learn_ranker
    guesses = taskllm.predict(prompts, augment=train.swap_refs).argmax(dim=1)
  File "/home/pmineiro/lampstuff/personalized_citation/TaskLLM.py", line 39, in predict
    return self.forward(x, augment=augment)
  File "/home/pmineiro/lampstuff/personalized_citation/TaskLLM.py", line 29, in forward
    augment_logprobs = self.forward(augment(data))[:,[1,0]]
  File "/home/pmineiro/lampstuff/personalized_citation/TaskLLM.py", line 22, in forward
    enc = self._tokenizer(data, return_tensors='pt', padding=True).to(self._transformer.device)
  File "/home/pmineiro/miniconda3/envs/lampstuff/lib/python3.10/site-packages/transformers/tokenization_utils_base.py", line 2602, in __call__
    encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)
  File "/

# Step 3: Prepare leaderboard submission files

In [ ]:
def prepare_submission(*, rank):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedCitation import train_loader, dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from peft import PeftConfig, IA3Config, TaskType
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    dev = dev_loader(batch_size=2, timesplit=True)
    test = test_loader(batch_size=2, timesplit=True)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter('Time_keq1_t5base_step1', 'taskllm')
    t5.load_adapter(f'Time_keq1_t5base_step2_rankeq{rank}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'Time_keq1_t5base_step2_rankeq{rank}')
    
    def reward_augment(inputs):
        import re
        return [ re.sub(r'Ref1: (.*)\nRef2: (.*)\nExtra:',
                        r'Ref1: \2\nRef2: \1\nExtra:',
                        z)
                 for z in inputs ]

    with ProgressPrinter(f'{rank} acc (dev)') as printer:
        devgolds = []
        testgolds = []
        for isdev, (examples, labels) in interleave(dev, test):
            for ex, label in zip(examples, labels):
                greedyrewards = []
                with torch.no_grad():
                    embeddings = dev.embed( [ ex['ref1'], ex['ref2'] ] + 
                                              [ v['title'] 
                                               for v in ex['profile']
                                               if v['title'] != ex['title'] 
                                             ])
                    scores = torch.max(embeddings[[0,1],:] @ embeddings[2:,:].T, dim=0).values
                    index = torch.topk(scores, dim=0, k=rank).indices.to('cpu')
                    prompts = []
                    rhatprompts = []
                    for n, oneind in enumerate(index.tolist()):
                        titles = [ f'{ex["profile"][ind]["title"]}' for ind in (oneind,) ]
                        concat_titles = ' and '.join([f'"{v}"' for v in titles])
                        input = dev.append_to_title(ex, concat_titles)
                        prompts.append(input)
                        rhatprompt = f"Title: {ex['title']}\nRef1: {ex['ref1']}\nRef2: {ex['ref2']}\nExtra: {titles[0]}"
                        rhatprompts.append(rhatprompt)

                    rhats = rewardpredictor.predict(rhatprompts, augment=reward_augment)
                    greedy = torch.argmax(rhats, dim=0).item()
                    guess = taskllm.predict([ prompts[greedy] ], augment=dev.swap_refs).argmax(dim=1)
                    if isdev:
                        target = int(label == dev.choices[1])
                        reward = float(guess == target)
                        greedyrewards.append(reward)

                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': "[2]" if guess else "[1]" })

            greedyacc = torch.Tensor(greedyrewards).float().mean().item() if isdev else None

            printer.addobs(greedyacc)

        printer.print()
        printer.autoprint = False

        for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
            with open(f'lamp1t_{wut}golds_rankeq{rank}.json', 'w') as jsonfile:
                json.dump({ 'task': 'LaMP_1', 'golds': golds }, jsonfile)
            
from Fork import SubProcess
with SubProcess() as process: process.parent or prepare_submission(rank=8)